In [ ]:
# pip install djitellopy
'''
pip install torch torchvision
git clone https://github.com/ultralytics/yolov5
cd yolov5
pip install -r requirements.txt
'''
'''
# 初始化 Tello 无人机
drone = Tello()
drone.connect()
drone.streamon()


# load YOLOv5 model
model = torch.hub.load('ultralytics/yolov5', 'custom', path='best.pt')
model.conf = 0.5
'''

In [ ]:
import torch

# 加载模型
model = torch.hub.load('ultralytics/yolov5', 'custom', path='best.pt')

# 查看标签
print("Label 映射:")
for i, label in model.names.items():
    print(f"{i}: {label}")

In [1]:
import cv2
import torch
from time import sleep
from djitellopy import Tello

class LogoHunting:
    def __init__(self, model_path):
        self.drone = Tello()
        sleep(2)
        self.drone.connect()
        self.drone.streamon()
        self.wait_for_camera_ready()

        self.model = torch.hub.load('ultralytics/yolov5', 'custom', path=model_path)
        self.model.conf = 0.5

        self.frame_width = 640
        self.frame_height = 480

    def take_off(self):
        response=self.drone.get_battery()
        print(response)
        sleep(2)
        
        try:
            print("[INFO] Sending takeoff command...")
            result = self.drone.takeoff()
            print("[INFO] Takeoff result:", result)
            sleep(2)
        except Exception as e:
            print("[ERROR] Takeoff failed:", str(e))
            try:
                self.drone.land()
            except:
                pass
    
    def land(self):
        self.drone.land()

    def wait_for_camera_ready(self, max_wait=5):
        print("[INFO] Waiting for camera to warm up...")
        for _ in range(max_wait * 10):  # 0.1s x 50 = 5s
            frame = self.get_camera_frame()
            if frame is not None and frame.shape[0] > 0:
                print("[INFO] Camera ready.")
                return True
            sleep(0.1)
        raise RuntimeError("Camera failed to initialize in time!")
    
    def shutdown(self):
        self.drone.streamoff()
        self.drone.end()

    def debug_camera_stream(self):
        print("[INFO] Starting camera debug stream... Press 'q' to exit.")
        while True:
            frame = self.get_camera_frame()
            cv2.imshow("Tello Camera", frame)

            if cv2.waitKey(1) & 0xFF == ord('q'):
                break

        cv2.destroyAllWindows()

    # get camera
    def get_camera_frame(self):
        frame = self.drone.get_frame_read().frame
        return cv2.resize(frame, (640, 480))

    # motion control 
    def rotate_drone(self,degree=30):
        self.drone.rotate_clockwise(int(degree))
        sleep(1)

    def move_forward(self,distance=20):
        self.drone.move_forward(int(distance))
        sleep(1)

    def turn_left(self,degree = 10):
        self.drone.rotate_counter_clockwise(int(degree))
        sleep(1)

    def turn_right(self,degree = 10):
        self.drone.rotate_clockwise(int(degree))
        sleep(1)

    def stop_drone(self):
        self.drone.send_rc_control(0, 0, 0, 0)

    # tools
    def get_box_center(self,box):
        x1, y1, x2, y2 = box
        return ((x1 + x2) / 2, (y1 + y2) / 2)

    def get_box_area_ratio(self,box):
        x1, y1, x2, y2 = box
        box_area = (x2 - x1) * (y2 - y1)
        return box_area / (self.frame_width * self.frame_height)

    # YOLO detect
    def yolo_detect(self,frame,max_retries = 1):
        retry_count = 0
        while retry_count <= max_retries:
            try:
                if frame is None or frame.size == 0:
                    raise ValueError("Empty frame")

                results = self.model(frame)
                boxes = results.xyxy[0].cpu().numpy()

                icon_box = None

                for box in boxes:
                    x1, y1, x2, y2, conf, cls = box
                    label = results.names[int(cls)].lower()
                    if 'icon' in label:
                        icon_box = [x1, y1, x2, y2]
                    #elif 'text' in label:
                        #text_box = [x1, y1, x2, y2]

                if icon_box is None:
                    raise ValueError("No objects detected")

                return {"icon_box": icon_box}

            except Exception as e:
                print(f"[WARN] Detection failed (retry {retry_count}): {e}")
                retry_count += 1
                sleep(1)
                frame = self.get_camera_frame()

        print("[ERROR] YOLO detection failed after retries.")
        return {"icon_box": None}

    # search for target
    # will turn 2 circles, trying to find the target
    # if find: return icon box; if not: return None
    def rotate_until_target_found(self):
        i = 0
        while i <= 36:
            if i >= 12:
                frame = self.get_camera_frame()
                result = self.yolo_detect(frame,max_retries = 0)
                if result["icon_box"]:
                    print("Find Target")
                    return result["icon_box"]
                sleep(0.5)
                if i == 12:
                    self.drone.move_up(20)
                if i == 24:
                    self.drone.move_down(40)
                #else:
                    #if i % 2 == 1:
                        #self.drone.move_down(20)
                    #else:
                        #self.drone.move_up(20)
            frame = self.get_camera_frame()
            result = self.yolo_detect(frame)
            if result["icon_box"]:
                print("Find Target")
                return result["icon_box"]
            self.rotate_drone()
            sleep(1)
            i += 1
        return None

    # fly towards icon
    # Set a max try round 
    # if reach the target, return True; if failed return False
    def fly_toward_icon(self,max_try_round = 50):
        for _ in range(max_try_round):
            frame = self.get_camera_frame()
            if frame is None:
                print("[WARNING] Frame is empty, skipping this round.")
                continue

            result = self.yolo_detect(frame,max_retries = 3)
            icon_box = result["icon_box"]

            if icon_box is None:
                print("[WARNING] Detection failed. Try 2 more frame before rotating.")
                for try_count in range(4):
                    sleep(0.5)
                    if try_count == 0:
                        self.drone.move_up(20)
                    if try_count == 1:
                        self.drone.move_down(40)
                    if try_count == 2:
                        self.drone.move_up(20)
                        self.turn_right(10)
                    else:
                        self.turn_left(20)
                    frame_retry = self.get_camera_frame()
                    result_retry = self.yolo_detect(frame_retry)
                    icon_box = result_retry["icon_box"]
                    if icon_box is not None:
                        break

                if icon_box is None:
                    self.stop_drone()
                    flag = 0
                    i = 0
                    while not flag and i <3:
                        print("Lost target, researching...")
                        flag = self.rotate_until_target_found()
                        i += 1
                    if not flag:
                        print("Can't find target! Maybe have some problems!") 
                        return False
                    else: icon_box = flag

            area_ratio = self.get_box_area_ratio(icon_box)
            print(f"Current icon box area ratio: {area_ratio:.4f}")

            if area_ratio > 0.25:
                self.stop_drone()
                print("Arrived!")
                return True

            x_center, y_center = self.get_box_center(icon_box)
            screen_center_x = self.frame_width / 2
            screen_center_y = self.frame_height / 2

            if abs(x_center - screen_center_x) > 80:  # need to test for the specific numbers
                if x_center > screen_center_x:
                    print("Turn right to face the icon")
                    if area_ratio < 0.1:
                        self.turn_right(max(10,(abs(x_center - screen_center_x))*area_ratio*2.5))
                    else:
                        self.drone.move_right(max(20,int(abs(x_center - screen_center_x)/5)))
                else:
                    print("Turn left to face the icon")
                    if area_ratio < 0.1:
                        self.turn_left(max(10,(abs(x_center - screen_center_x))*area_ratio*2.5))
                    else:
                        self.drone.move_left(max(20,int(abs(x_center - screen_center_x)/5)))
            if abs(y_center - screen_center_y) > 60:  # need to test for the specific numbers
                if y_center < screen_center_y:
                    print("Flying too low! Get up!")
                    self.drone.move_up(max(20,int(area_ratio*abs(y_center - screen_center_y)*0.8)))
                else:
                    print("Flying too high! Get down!")
                    self.drone.move_down(max(20,int(area_ratio*abs(y_center - screen_center_y)*0.8)))
                   
            
            print("Move Towards the Icon!")
            self.move_forward(max(20,int(50-area_ratio*120)))

        return False
            
    # read through the text and show off
    def side_slide_to_text(self):
        print("Begin reading...")

        """
        while True:
            self.drone.move_right(20)  # flying first!
            sleep(1.5)

            frame = self.get_camera_frame()
            result = self.yolo_detect(frame)
            text_box = result["text_box"]

            if text_box is None:
                self.stop_drone()
                print("lose target...")
                self.rotate_until_target_found()
                continue

            x1, y1, x2, y2 = text_box
            # if reach the right end
            if x2 > 0.9 * 640:
                self.stop_drone()
                print("Finish reading!")
                break        
        """
        for i in range(4):      # Hardcoding
            self.drone.move_right(30)  
            sleep(1)


        print("Turn to the audience")
        self.drone.rotate_clockwise(180)
        sleep(2)
        return True
        


In [ ]:
import os
os.system("pkill -f python")
mission = LogoHunting(model_path='best.pt')

mission.take_off()

flag = mission.rotate_until_target_found()
print(flag)

if flag:
    flag = False
    flag = mission.fly_toward_icon()
    print(flag)

    if flag:
        mission.side_slide_to_text()

mission.shutdown()
        

In [ ]:
mission.shutdown()

In [2]:
import os
os.system("pkill -f python")
mission = LogoHunting(model_path='best.pt')
#mission.debug_camera_stream()

[INFO] tello.py - 129 - Tello instance was initialized. Host: '192.168.10.1'. Port: '8889'.
[INFO] tello.py - 438 - Send command: 'command'
[INFO] tello.py - 462 - Response command: 'ok'
[INFO] tello.py - 438 - Send command: 'streamon'
[INFO] tello.py - 462 - Response streamon: 'ok'


[INFO] Waiting for camera to warm up...
[INFO] Camera ready.


Using cache found in /Users/wecky/.cache/torch/hub/ultralytics_yolov5_master
left block unavailable for requested intra mode
error while decoding MB 0 34, bytestream 1446
YOLOv5 🚀 2025-4-5 Python-3.11.3 torch-2.6.0 CPU

Fusing layers... 
Model summary: 157 layers, 7015519 parameters, 0 gradients, 15.8 GFLOPs
Adding AutoShape... 


In [3]:
mission.take_off()

44


[INFO] tello.py - 438 - Send command: 'takeoff'


[INFO] Sending takeoff command...


[INFO] tello.py - 462 - Response takeoff: 'ok'


[INFO] Takeoff result: None


error while decoding MB 55 40, bytestream -6


In [ ]:
mission.drone.move_left(20)


In [ ]:
#mission.turn_left(10)

In [ ]:
"""
mission.drone.move_up(20)
sleep(2)
mission.drone.move_down(15)
"""


In [ ]:
"""
mission.drone.move_right(20)
sleep(2)
mission.drone.land()
"""


In [4]:
flag = mission.rotate_until_target_found()
print(flag)

/Users/wecky/.cache/torch/hub/ultralytics_yolov5_master/models/common.py:906: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with amp.autocast(autocast):


[WARN] Detection failed (retry 0): No objects detected


left block unavailable for requested intra4x4 mode -1
error while decoding MB 0 20, bytestream 10610
/Users/wecky/.cache/torch/hub/ultralytics_yolov5_master/models/common.py:906: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with amp.autocast(autocast):
error while decoding MB 26 39, bytestream -6


[WARN] Detection failed (retry 1): No objects detected


[INFO] tello.py - 438 - Send command: 'cw 30'


...

[ERROR] YOLO detection failed after retries.


[INFO] tello.py - 462 - Response cw 30: 'ok'
/Users/wecky/.cache/torch/hub/ultralytics_yolov5_master/models/common.py:906: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with amp.autocast(autocast):


[WARN] Detection failed (retry 0): No objects detected
[ERROR] YOLO detection failed after retries.
Find Target
[1.3383827, 99.34325, 106.32712, 182.28468]


/Users/wecky/.cache/torch/hub/ultralytics_yolov5_master/models/common.py:906: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with amp.autocast(autocast):


error while decoding MB 47 34, bytestream -6
left block unavailable for requested intra mode
error while decoding MB 0 35, bytestream 1479


In [5]:
flag = mission.fly_toward_icon()
print(flag)

/Users/wecky/.cache/torch/hub/ultralytics_yolov5_master/models/common.py:906: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with amp.autocast(autocast):
error while decoding MB 18 41, bytestream -6


[WARN] Detection failed (retry 0): No objects detected


/Users/wecky/.cache/torch/hub/ultralytics_yolov5_master/models/common.py:906: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with amp.autocast(autocast):
[INFO] tello.py - 438 - Send command: 'ccw 18'


Current icon box area ratio: 0.0277
Turn left to face the icon


left block unavailable for requested intra mode
error while decoding MB 0 28, bytestream 1785
[INFO] tello.py - 462 - Response ccw 18: 'ok'
[INFO] tello.py - 438 - Send command: 'up 20'


...

[INFO] tello.py - 438 - Send command: 'cw 30'
[INFO] tello.py - 462 - Response cw 30: 'error Auto land'
[INFO] tello.py - 438 - Send command: 'cw 30'


[ERROR] YOLO detection failed after retries.


[INFO] tello.py - 462 - Response cw 30: 'error Auto land'
[INFO] tello.py - 438 - Send command: 'cw 30'
[INFO] tello.py - 462 - Response cw 30: 'error Auto land'


TelloException: Command 'cw 30' was unsuccessful for 4 tries. Latest response:	'error Auto land'

error while decoding MB 4 43, bytestream -6


In [ ]:
mission.side_slide_to_text()

In [6]:
#mission.land()
mission.shutdown()

[INFO] tello.py - 438 - Send command: 'streamoff'
[INFO] tello.py - 462 - Response streamoff: 'ok'
[INFO] tello.py - 438 - Send command: 'land'
[INFO] tello.py - 462 - Response land: 'ok'


In [ ]:
print("[INFO] Battery:", mission.drone.get_battery())
print("[INFO] Height:", mission.drone.get_height())